### Run in collab
<a href="https://colab.research.google.com/github/racousin/data_science_practice/blob/master/website/public/modules/data-science-practice/module9/exercise/module9_exercise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install swig==4.2.1
!pip install gymnasium==1.2.0

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import random, json

In [ ]:
# Petit test pour voir combien d'actions et d'états il y a dans FrozenLake 8x8

env = gym.make("FrozenLake-v1", map_name="8x8")

print("nb de cases :", env.observation_space)
print("nb de mouvements:", env.action_space)

env.close()


# module9_exercise2 : ML - Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a>

### Objective
Get at list an agent running on ML-Arena <a href="https://ml-arena.com/viewcompetition/5" target="_blank"> FrozenLake Competition</a> with mean reward upper than 0.35 (ie 35%)


You should submit an agent file named `agent.py` with a class `Agent` that includes at least the following attributes:

In [ ]:
class Agent:
    def __init__(self, env):
        self.env = env

    def choose_action(self, observation, reward=0.0, terminated=False, truncated=False, info=None):
        action = self.env.action_space.sample() # your logic here
        return action

### Description

The game starts with the player at location [0,0] of the frozen lake grid world with the goal located at far extent of the world [7,7].

Holes in the ice are distributed in set locations.

The player makes moves until they reach the goal or fall in a hole.

Each run will consist of 10 attempts to cross the ice. The reward will be the total amount accumulated during those trips. For example, if your agent reaches the goal 3 times out of 10, its reward will be 3.

The environment is based on :

In [ ]:
env = gym.make('FrozenLake-v1', map_name="8x8")

In [ ]:


def entrainer_agent(nb_episodes=50000, alpha=0.1, gamma=0.99, eps_debut=1.0, eps_fin=0.01, pas_reduction=200000):
    #je prends 1 pour l'epsilon du début pour que l'agent explore tout et à la fin je prends 0.01 pour qu'il exploite ce qu'il a appris
    #je prends ce nombre d'épisodes pour bcp apprendre ?
    #je prends un petit alpha pour que la vitesse d'apprentissage soit fiable
    #je prends gamma=0.99 pour que l'agent anticipe
    #et les pas_reduction c le nb d'episodes pendant lesquels on fait baisser epsilon
    env = gym.make("FrozenLake-v1", map_name="8x8")   # je créée l'environement du lac gelé
    nb_etats = env.observation_space.n                # c'était déjà marqué, mais comme on a 8x8 j'ai 64 cases
    nb_actions = env.action_space.n                   # j'ai fait un bloc après le bloc des import pour vérifier et on peut aller dans 4 directions : à droite, à gauche, en haut, en bas

    #on remplie Q de 0 au début pout initialiser
    Q = np.zeros((nb_etats, nb_actions))
    epsilon = eps_debut  #pcq du coup on explore au début donc tout au hasard

    # je joue plein de parties
    for episode in range(nb_episodes):
        etat, _ = env.reset(seed=episode)  # on démarre au début de la grille
        termine = faux = False  #je démarre au départ du lac, pour l'instant la partie n'est pas finie

        #je continue tant que je suis pas tombée dans un trou ou tant que j'ai pas fini la partie
        while not (termine or faux):
            #là je vais choisir : soit je prends le meilleur score, soit je décide au hasard
            if random.random() < epsilon:
                action = env.action_space.sample()   # au hasard
            else:
                meilleures = np.flatnonzero(Q[etat] == Q[etat].max())  # on prend l'action qui a le meilleur score
                action = int(random.choice(meilleures))  # si ça se trouve j'ai plusieurs ex aequo donc je choisis au hasard, c pg c tous le meme score

            nouvel_etat, recompense, termine, faux, _ = env.step(action)

            #la je mets à jour la Q table grace à ce que j'ai fait avant
            meilleure_suite = Q[nouvel_etat].max() #si je continue depuis ça, quelle est la meilleure recompense possible ?
            cible = recompense + (0 if (termine or faux) else gamma * meilleure_suite) #donc la c la recompense d'ajd + ce que je pourrais encore gagner demain
            # mise à jour avec alpha (mélange entre ancienne valeur et nouvelle info)
            Q[etat, action] += alpha * (cible - Q[etat, action]) #j'ignore pas complètement l'ancienne valeur, peut-être que si non je serai trop influencée par un seul essai

            etat = nouvel_etat  #j'avance dans la partie

        # au fur et à mesure, je réduis epsilon, pour ne plus choisir 100% au hasard mais explorer ce que j'ai déjà appris
        if episode < pas_reduction:
            epsilon = eps_debut - (eps_debut - eps_fin) * (episode / pas_reduction)
        else:
            epsilon = eps_fin

    env.close()
    return Q


def tester_agent(Q, nb_runs=200, essais_par_run=10):
   #là je teste juste pour voir si ça marche bien
    env = gym.make("FrozenLake-v1", map_name="8x8")
    total_runs = []
    for r in range(nb_runs):
        total = 0
        for _ in range(essais_par_run): #à chaque essai je remets le joueur en haut à gauche
            etat, _ = env.reset()
            termine = faux = False
            while not (termine or faux):   #je joue la partie jusqu'à la fin
                # à chaque état, je choisis l'action qui a le meilleur score dans Q
                meilleures = np.flatnonzero(Q[etat] == Q[etat].max())
                action = int(random.choice(meilleures))
                etat, recompense, termine, faux, _ = env.step(action)
                total += recompense
        total_runs.append(total)
    env.close()

    moyenne_run = np.mean(total_runs)                  # reward moyen
    taux_succes = moyenne_run / essais_par_run         # taux de succès en pourcentage
    return moyenne_run, taux_succes


#je teste et on va faire la moyenne
Q = entrainer_agent()
moyenne, succes = tester_agent(Q)
print("reward moyenne :", round(moyenne, 2))
print("succès moyen  :", round(succes, 3), "≈", round(100*succes,1), "%")



In [ ]:
#donc la je mets la Q table dans mon agent.py
q_list = np.asarray(Q, dtype=float).round(6).tolist()

agent_code = f"""
import random

class Agent:
    def __init__(self, env):
        # je garde l'environnement
        self.env = env
        # Ici on colle la Q-table apprise hors ligne
        self.Q = {json.dumps(q_list)}
        self.nS = len(self.Q)                   # 64 états pour 64 cases
        self.nA = len(self.Q[0]) if self.nS>0 else env.action_space.n  # 4 actions : en haut, en bas, à droite, à gauche

    def choose_action(self, observation, reward=0.0, terminated=False, truncated=False, info=None):
        # on observe la case
        s = int(observation)

        #si y a un pb je mets une action au hasard
        if s < 0 or s >= self.nS:
            return self.env.action_space.sample()

        # je récupère la ligne de la Q-table pour cet état
        row = self.Q[s]
        # et je cherche la meilleure valeur Q dans cette ligne
        m = max(row)
        # je prends toutes les actions qui atteignent ce max
        meilleures = [a for a, q in enumerate(row) if q == m]
        # je choisis au hasard parmi les meilleures si plusieurs ont le meme max
        return random.choice(meilleures)
"""

#et la je dois mettre agent.py sur ml arena
with open("agent.py", "w") as f:
    f.write(agent_code)




In [ ]:
from google.colab import files
files.download("agent.py")


### Before submit
Test that your agent has the right attributes

In [ ]:
#je veux vérifier que agent.py marche bien
env = gym.make('FrozenLake-v1', map_name="8x8")

#j'importe ma classe d'agent
from agent import Agent


agent = Agent(env)

# je vérifie si l'agent joue bien avec une partie
observation, _ = env.reset()  #ça c état de départ
reward, termine, faux, info = None, False, False, None
rewards = []

while not (termine or faux):  # pareil tant que la partie n’est pas finie
    #en se basant sur la q table l'agent choisit une action
    action = agent.choose_action(
        observation,
        reward=reward,
        terminated=termine,
        truncated=faux,
        info=info
    )
    # je joue cette action dans l’environnement
    observation, reward, termine, faux, info = env.step(action)
    rewards.append(reward)

#pour afficher le score total
print(f"reward sur un seul essai : {sum(rewards)}")

#OK CA MARCHE PCQ JAI UN REWARD DE 1.0 SUR UN SEUL ESSAI YOUPIIIIIII
